<a href="https://colab.research.google.com/github/leonardoleandrodev/covidMachineLearningPrediction/blob/main/DEEP_LEARNING_KERAS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from numpy import loadtxt
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from pandas import read_csv as rc,factorize as fc,to_numeric as tnum,set_option as op

In [ ]:
# dados do site https://opendatasus.saude.gov.br/dataset/bd-srag-2021
urlfile = 'https://opendatasus.saude.gov.br/dataset/9f76e80f-a2f1-4662-9e37-71084eae23e3/resource/42bd5e0e-d61a-4359-942e-ebc83391a137/download/influd21-18-06-2021.csv'
#https://opendatasus.saude.gov.br/dataset/9f76e80f-a2f1-4662-9e37-71084eae23e3/resource/b3321e55-24e9-49ab-8651-29cf5c8f3179/download/dicionario_de_dados_srag_hospitalizado_23.03.2021.pdf
dataset = rc(urlfile, delimiter=';',encoding='ISO-8859-1')


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (15,18,60,62,63,64,92,94,106,108,115,117,118,119,123,144,160) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
op('display.max_columns', None)
dataset.head(10)


,DT_NOTIFIC,SEM_NOT,DT_SIN_PRI,SEM_PRI,SG_UF_NOT,ID_REGIONA,CO_REGIONA,ID_MUNICIP,CO_MUN_NOT,ID_UNIDADE,CO_UNI_NOT,CS_SEXO,DT_NASC,NU_IDADE_N,TP_IDADE,COD_IDADE,CS_GESTANT,CS_RACA,CS_ETINIA,CS_ESCOL_N,ID_PAIS,CO_PAIS,SG_UF,ID_RG_RESI,CO_RG_RESI,ID_MN_RESI,CO_MUN_RES,CS_ZONA,SURTO_SG,NOSOCOMIAL,AVE_SUINO,FEBRE,TOSSE,GARGANTA,DISPNEIA,DESC_RESP,SATURACAO,DIARREIA,VOMITO,OUTRO_SIN,OUTRO_DES,PUERPERA,FATOR_RISC,CARDIOPATI,HEMATOLOGI,SIND_DOWN,HEPATICA,ASMA,DIABETES,NEUROLOGIC,PNEUMOPATI,IMUNODEPRE,RENAL,OBESIDADE,OBES_IMC,OUT_MORBI,MORB_DESC,VACINA,DT_UT_DOSE,MAE_VAC,DT_VAC_MAE,M_AMAMENTA,DT_DOSEUNI,DT_1_DOSE,DT_2_DOSE,ANTIVIRAL,TP_ANTIVIR,OUT_ANTIV,DT_ANTIVIR,HOSPITAL,DT_INTERNA,SG_UF_INTE,ID_RG_INTE,CO_RG_INTE,ID_MN_INTE,CO_MU_INTE,UTI,DT_ENTUTI,DT_SAIDUTI,SUPORT_VEN,RAIOX_RES,RAIOX_OUT,DT_RAIOX,AMOSTRA,DT_COLETA,TP_AMOSTRA,OUT_AMOST,PCR_RESUL,DT_PCR,POS_PCRFLU,TP_FLU_PCR,PCR_FLUASU,FLUASU_OUT,PCR_FLUBLI,FLUBLI_OUT,POS_PCROUT,PCR_VSR,PCR_PARA1,PCR_PARA2,PCR_PARA3,PCR_PARA4,PCR_ADENO,PCR_METAP,PCR_BOCA,PCR_RINO,PCR_OUTRO,DS_PCR_OUT,CLASSI_FIN,CLASSI_OUT,CRITERIO,EVOLUCAO,DT_EVOLUCA,DT_ENCERRA,DT_DIGITA,HISTO_VGM,PAIS_VGM,CO_PS_VGM,LO_PS_VGM,DT_VGM,DT_RT_VGM,PCR_SARS2,PAC_COCBO,PAC_DSCBO,OUT_ANIM,DOR_ABD,FADIGA,PERD_OLFT,PERD_PALA,TOMO_RES,TOMO_OUT,DT_TOMO,TP_TES_AN,DT_RES_AN,RES_AN,POS_AN_FLU,TP_FLU_AN,POS_AN_OUT,AN_SARS2,AN_VSR,AN_PARA1,AN_PARA2,AN_PARA3,AN_ADENO,AN_OUTRO,DS_AN_OUT,TP_AM_SOR,SOR_OUT,DT_CO_SOR,TP_SOR,OUT_SOR,DT_RES,RES_IGG,RES_IGM,RES_IGA,ESTRANG,VACINA_COV,DOSE_1_COV,DOSE_2_COV,LAB_PR_COV,LOTE_1_COV,LOTE_2_COV,FNT_IN_COV
0,06/01/2021,1,05/01/2021,1,SP,GVE XII ARARAQUARA,1337.0,ITAPOLIS,352270,SANTA CASA DE MISERICORDIA ITAPOLIS,2079836,M,26/03/1953,67,3,3067,6,2,NaN,NaN,BRASIL,1,SP,GVE XII ARARAQUARA,1337.0,ITAPOLIS,352270.0,1.0,NaN,NaN,NaN,2.0,1.0,2.0,2.0,2.0,2.0,9.0,1.0,2.0,NaN,2.0,1,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,NaN,2.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,1.0,05/01/2021,SP,GVE XII ARARAQUARA,1337.0,ITAPOLIS,352270.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,19/01/2021,4.0,SANGUE,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,1.0,1.0,28/01/2021,28/01/2021,06/01/2021,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,19/01/2021,1.0,NaN,19/01/2021,2.0,2.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,09/01/2021,1,08/01/2021,1,MG,UBERLANDIA,1462.0,UBERLANDIA,317020,UAI SAO JORGE,5617286,M,07/05/2002,18,3,3018,6,2,NaN,9.0,BRASIL,1,MG,UBERLANDIA,1462.0,UBERLANDIA,317020.0,1.0,2.0,2.0,2.0,1.0,1.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,CRISE CONVULSIVA,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,1.0,09/01/2021,MG,UBERLANDIA,1462.0,UBERLANDIA,317020.0,2.0,NaN,NaN,3.0,6.0,NaN,NaN,1.0,10/01/2021,1.0,NaN,1.0,10/01/2021,2.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,1.0,1.0,13/01/2021,13/01/2021,10/01/2021,2,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,2.0,2.0,2.0,2.0,6.0,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10/01/2021,2,05/01/2021,1,SP,GVE XXX JALES,1574.0,JALES,352480,SANTA CASA DE MISERICORDIA DE JALES,2079895,M,15/10/1963,57,3,3057,6,1,NaN,9.0,BRASIL,1,SP,GVE XXX JALES,1574.0,JALES,352480.0,1.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,NaN,2.0,1,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,NaN,1.0,HIPERTENSAO ARTERIAL,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,1.0,NaN,SP,GVE XXX JALES,1574.0,JALES,352480.0,2.0,NaN,NaN,3.0,6.0,NaN,NaN,1.0,10/01/2021,1.0,NaN,2.0,15/01/2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11/01/2021,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,2.0,1.0,1.0,NaN,10/01/2021,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10/01/2021,2,08/01/2021,1,RS,002 CR

In [ ]:
amostra = dataset[
                  ['CS_SEXO','NU_IDADE_N','FEBRE','TOSSE',
                   'GARGANTA','DIARREIA','VOMITO','OUTRO_SIN',
                   'CARDIOPATI','HEMATOLOGI','HEPATICA','ASMA',
                   'DIABETES','PNEUMOPATI','RENAL','OBESIDADE',
                   'ANTIVIRAL','UTI','DOR_ABD','FADIGA'
                   ,'PERD_OLFT','PERD_PALA','EVOLUCAO','CLASSI_FIN']]#[1:10000]
amostra = amostra.dropna()
#amostra['IdadeNaDataNotificacao'] = [int(idade.split(' ')[0]) for idade in amostra['IdadeNaDataNotificacao']]
#amostra['IdadeNaDataNotificacao'] = [int(idade.split(' ')[0]) for idade in amostra['IdadeNaDataNotificacao']]
#amostra = amostra.drop(amostra[amostra.ResultadoTesteRapido == 'Não Informado'].index)
#amostra = amostra.drop(amostra[amostra.ResultadoTesteRapido == 'Inconclusivo'].index)
#amostra = amostra.drop(amostra[amostra.ComorbidadePulmao == '-'].index)
#amostra = amostra.drop(amostra[amostra.RacaCor == 'Ignorado'].index)
amostra = amostra.drop(amostra[amostra.NU_IDADE_N < 20 ].index)
amostra = amostra.drop(amostra[amostra.EVOLUCAO > 2 ].index)
amostra = amostra.drop(amostra[amostra.FEBRE > 2 ].index)
amostra = amostra.drop(amostra[amostra.TOSSE > 2 ].index)
amostra = amostra.drop(amostra[amostra.GARGANTA > 2 ].index)
amostra = amostra.drop(amostra[amostra.FADIGA > 2 ].index)
amostra = amostra.drop(amostra[amostra.PERD_OLFT > 2 ].index)
amostra = amostra.drop(amostra[amostra.PERD_PALA > 2 ].index)
amostra = amostra.drop(amostra[amostra.UTI > 2 ].index)
#amostra = amostra.drop(amostra[amostra.Classificacao == 'Suspeito' ].index)
#amostra = amostra.drop(amostra[amostra.Evolucao == 'Ignorado'].index)
#amostra = amostra.drop(amostra[amostra.Evolucao == '-'].index)
#amostra = amostra.drop(amostra[amostra.Evolucao == 'Óbito por outras causas'].index)

amostra.head()


,CS_SEXO,NU_IDADE_N,FEBRE,TOSSE,GARGANTA,DIARREIA,VOMITO,OUTRO_SIN,CARDIOPATI,HEMATOLOGI,HEPATICA,ASMA,DIABETES,PNEUMOPATI,RENAL,OBESIDADE,ANTIVIRAL,UTI,DOR_ABD,FADIGA,PERD_OLFT,PERD_PALA,EVOLUCAO,CLASSI_FIN
0,M,67,2.0,1.0,2.0,9.0,1.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,4.0
3,F,83,2.0,2.0,2.0,2.0,2.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,9.0,2.0,2.0,2.0,2.0,2.0,1.0,4.0
4,M,62,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,5.0
5,F,63,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,5.0
6,F,64,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,1.0,5.0


In [ ]:
colunas = ['FADIGA','PERD_OLFT','PERD_PALA','CLASSI_FIN','UTI']
classe = colunas[-1]
amostrax = amostra[colunas]

amostrax['CLASSI_FIN'] = amostrax['CLASSI_FIN']#.apply(lambda x: 1 if x == 5.0 else 0)
#amostrax['CS_SEXO'] = amostrax['CS_SEXO'].replace('F',0).replace('M',1)
#amostrax = amostrax[[c for c in colunas]]
#amostrax['NU_IDADE_N'] = [round(idd/10)*10 for idd  in amostrax['NU_IDADE_N']]
dados = amostrax.replace(1.0,1).replace(2.0,0)

#dados[colunas] = dados[colunas].astype(int)
del dados[classe]
resnegposi = dados[colunas[:-1]].sum(axis=1).apply(lambda x: 1 if x > 6.0 else 0)
#amostrax[[classe]].replace(1.0,1).replace(2.0,0)#.replace('Descartados',0).replace('Confirmados',1).replace('Negativo',0).replace('Positivo',1)
dados[20:].head(10) 
resnegposi.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


0    0
3    0
4    0
5    0
6    0
dtype: int64

In [ ]:

dadosX = dados.apply(lambda col: fc(col)[0] if col.name == 'CS_SEXO' else col)
dadosX = dadosX.apply(tnum)
resnegposiY = resnegposi#.apply(lambda col: fc(col)[0])

#resnegposiY.info()

In [ ]:
from sklearn.model_selection import train_test_split
val_treino,val_teste,val_res_treino,val_res_teste = train_test_split(dadosX,resnegposiY,test_size=0.001,train_size=0.01)
(len(val_treino),len(val_teste))

(2032, 204)

In [ ]:
# definir o modelo com keras
# inicializar o modelo sequencial
model = Sequential()
# inicializar a primeira camada, com 12 neurÃ´nios, 8 entradas utilizando a funÃ§Ã£o ReLU
model.add(Dense(7, input_dim=(len(val_treino.columns)), activation='relu'))
# inicializar a segunda camada com 8 neurÃ´nios e a funÃ§Ã£o ReLU
#model.add(Dense(20, activation='relu'))
#model.add(Dense(20, activation='relu'))
#model.add(Dense(len(val_treino.columns), activation='relu'))
# inicializar a Ãºltima camada (camada de saÃ­da) com um neurÃ´nio e a funÃ§Ã£o Sigmoid
model.add(Dense(1, activation='sigmoid'))

In [ ]:
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
# treinar o modelo keras

model.fit(val_treino, val_res_treino, epochs=150, batch_size=10,use_multiprocessing=False,verbose=1)


Epoch 1/150
204/204 [==============================] - 1s 1ms/step - loss: 0.5055 - accuracy: 0.9071
Epoch 2/150
204/204 [==============================] - 0s 1ms/step - loss: 0.3394 - accuracy: 0.9079
Epoch 3/150
204/204 [==============================] - 0s 1ms/step - loss: 0.2959 - accuracy: 0.9068
Epoch 4/150
204/204 [==============================] - 0s 1ms/step - loss: 0.2855 - accuracy: 0.9032
Epoch 5/150
204/204 [==============================] - 0s 1ms/step - loss: 0.2654 - accuracy: 0.9053
Epoch 6/150
204/204 [==============================] - 0s 1ms/step - loss: 0.2631 - accuracy: 0.8974
Epoch 7/150
204/204 [==============================] - 0s 1ms/step - loss: 0.2381 - accuracy: 0.9043
Epoch 8/150
204/204 [==============================] - 0s 1ms/step - loss: 0.2284 - accuracy: 0.8942
Epoch 9/150
204/204 [==============================] - 0s 1ms/step - loss: 0.1782 - accuracy: 0.9111
Epoch 10/150
204/204 [==============================] - 0s 1ms/step - loss: 0.1616 - accura

In [ ]:
predictions = model.predict(val_teste)
sum = 0.0
for k in predictions:
  sum += k

md = (sum /len(predictions))-0.05
print(md)
predictionsi = [0 if p < 0.5 else 1 for p in predictions]
from sklearn.metrics import confusion_matrix,accuracy_score
accuracy = accuracy_score(val_res_teste, predictionsi)
mt = confusion_matrix(val_res_teste, predictionsi)
print(mt,f'acuracia de {accuracy} %')
for i,x in enumerate(predictionsi):
  if x != val_res_teste.values[i]:
    print(val_teste.values[i][0:],predictions[i],x,val_res_teste.values[i])
  #print(f'data={val_teste.values[i]} {predictions[i]}% {val_res_teste.values[i]} {x}')


[0.03549984]
[[186   2]
 [  0  16]] acuracia de 0.9901960784313726 %
[0. 1. 1. 4.] [0.96899164] 1 0
[1. 0. 1. 4.] [0.96899164] 1 0
